In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.usetex'] = True
mpl.rcParams['figure.dpi'] = 250

If you load this notebook from within the `examples` directory, you'll have to append the path before `import ggce`. Otherwise, this cell below will not be necessary. You can safely run it either way.

In [ ]:
import sys

try:
    import ggce
except ModuleNotFoundError:
    sys.path.append("..")
    import ggce

# Generalized Green's function Cluster Expansion (GGCE) intro API tutorial

The `GGCE` package is intended as a platform for solving zero and finite-temperature lattice model Hamiltonians within the Generalized Green's function Cluster Expansion method, explained in this [arXiv preprint](https://arxiv.org/pdf/2103.01972.pdf) and published work in Physical Review B (COMING SOON). It is based on the original work of Mona Berciu and coworkers, in which they developed the so-called Momentum Average (MA) Approximation, which leverages the ansatz that many electron-phonon models of interest create phonons in clouds. See for example [Berciu, PRL 2006](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.97.036402?casa_token=Qgfh9yLpKxoAAAAA%3AuLUpBffSY1sXarw47zqfPH6zE-cz31KHst66aLjrRRRcbjocbcoWmrfGgWXsqWN0iXFSft27KkQy) and [Berciu \& Fehske, PRB 2010](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.82.085116?casa_token=UvmWaAZ2HdgAAAAA%3Az60SeGb-LgHaj9FrHDCyl_hHlJ9TNc3Vkoa8KcGMWcD7U_hc1ZF5KzJEb4neJsfQFdjIdi8WDJy7).

## Theory
TODO

## Example $T=0$ calculations

We demonstrate the GGCE method on an example from [Berciu \& Fehske, PRB 2010](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.82.085116?casa_token=UvmWaAZ2HdgAAAAA%3Az60SeGb-LgHaj9FrHDCyl_hHlJ9TNc3Vkoa8KcGMWcD7U_hc1ZF5KzJEb4neJsfQFdjIdi8WDJy7). Specifically, we reproduce the results in Figure 5 (middle) for $k=\pi/2.$ This literature data has been manually ripped from the paper using [WebPlotDigitizer](https://apps.automeris.io/wpd/).

In [ ]:
literature_data = np.loadtxt("000_example_A.txt")

First, we import the `SerialSparseExecutor`, which will take care of constructing the linear system and solving it in sparse representation. We also import the `SerialDenseExecutor`, which will perform the same solve but in dense representation.

In [ ]:
from ggce.executors import SerialSparseExecutor, SerialDenseExecutor

The `system_params` dictionary varies significantly depending on the type of calculation desired, but follows the same general principles. The following five elements are all lists.
* `model` indicates the types of couplings in the electron-phonon Hamiltonian. For example, `["H"]` corresponds to a Holstein coupling term, $V = g \sum_i c_i^\dagger c_i (b_i^\dagger + b_i).$ `["H", "SSH"]` corresponds to a two-phonon mode coupling, in which the phonon operators are distinguishable for each coupling mode. Note that this doubles the Hilbert space size and therefore significantly increases the computational complexity of the calculation.
* `M_extent` indicates the maximum boson cloud size. For example, if $M=3,$ terms like $b_i^\dagger (b_{i+2}^\dagger)^2$ would be retained in the equation of motion, but terms like $b_i^\dagger b_{i+1}^\dagger b_{i+3}^\dagger$ would not.
* `N_bosons` indicates the maximum number of phonons allowed in any auxiliary equation. Any terms containing an auxiliary equation with more than that many phonons has coefficient set to zero, and is not included.
* `Omega` is the Einstein phonon frequencies.
* `dimensionless_coupling` is the dimensionles coupling term, which fixes `g` and depends on the model type.

Note that the length of all the provided lists above should **equal the total number of terms in the `model` list**. For example in the case of `["H", "SSH"]`, the first element of `Omega` is the phonon frequency corresponding to the Holstein term, and the second corresponds to the SSH term.

In [ ]:
system_params = {
    "model": ["EFB"],
    "M_extent": [3],
    "N_bosons": [9],
    "Omega": [1.25],
    "dimensionless_coupling": [2.5],
    "hopping": 0.1,
    "broadening": 0.005,
    "protocol": "zero temperature"
}

executor_dense = SerialDenseExecutor(system_params, "info")
executor_dense.prime()

executor_sparse = SerialSparseExecutor(system_params, "info")
executor_sparse.prime()

In [ ]:
wgrid = np.linspace(-5.5, -2.5, 100)

spectrum_sparse = [executor_sparse.solve(0.5 * np.pi, w) for w in wgrid]
spectrum_sparse = np.array([-s[0].imag / np.pi for s in spectrum_sparse])

spectrum_dense = [executor_dense.solve(0.5 * np.pi, w) for w in wgrid]
spectrum_dense = np.array([-s[0].imag / np.pi for s in spectrum_dense])

We can plot the results and show clearly that the results are the same, and barring normalizations due to different grid spacings, identical to the literature results.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))
ax.plot(wgrid, spectrum_sparse / spectrum_sparse.max(), 'k')
ax.plot(wgrid, spectrum_dense / spectrum_dense.max(), 'b--')
ax.plot(literature_data[:, 0], literature_data[:, 1], 'r--', linewidth=0.5)
ax.set_ylabel("$A(\pi/2, \omega) / \max A(\pi/2, \omega)$")
ax.set_xlabel("$\omega$")
plt.show()